In [2]:
import pandas as pd
import numpy as np
import re
import math
import tabula
import camelot
import pdfminer
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
try:
    from cStringIO import StringIO
except ImportError:
    from io import StringIO 
import matplotlib.pyplot as plt 

In [88]:
f = r"C:\Users\MudraCircle\Desktop\bks_raw\Parsing_testing\ICICI\files\garg_icici\icici04.pdf"
tables=tabula.read_pdf(f,
                          lattice=True,
                          pages="all",
                          silent=True,
                          multiple_tables=True,
                          pandas_options={'header':None})
df = pd.DataFrame()
try:
    df = pd.concat([c for c in tables]).drop_duplicates()
except:pass

# df = df.replace(r'^\s*$', np.nan, regex=True)
df=df[ df.isnull().sum(axis=1) < df.shape[1]-2].reset_index(drop=True)

try:
    idx=[c for c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('balance').any(),axis=1)==True].index if c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('date').any(),axis=1)==True].index][0]
    df.columns=df.iloc[idx]; df=df.iloc[idx+1:,:]; df.reset_index(drop=True, inplace=True)
except:
    try:
        idx=[c for c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('balance').any(),axis=1)==True].index if c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('description').any(),axis=1)==True].index][0]
        df.columns=df.iloc[idx]; df=df.iloc[idx+1:,:]; df.reset_index(drop=True, inplace=True)
    except:
        print("\nICICI Column Headers Missing"); pass
    
try:
    idx2 = [c for c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('page total').any(),axis=1)==True].index][0]    
    df.drop(df.index[idx2+1:], inplace=True)
except:pass

try:
    df = df.loc[:, df.columns.notnull()]
except:pass

try:
    bal=[c for c in df.columns if "BALANCE" in str(c).upper() ][0]
except: print("\nBalance columns missing")

In [89]:
# df.head(20)

In [90]:
def abc(a):
    if type(a) ==str:
        if len(a.split(' '))==2:
            z=a.split(' ')[1]
        else:
            z=a.split(' ')[0]
    else :
        z=a
    return z

try:
    bal=[c for c in df.columns if "BALANCE" in str(c).upper() ][0]
    df["bal"]=df[bal].apply( lambda x: abc(x) )
except: print("\nBalance columns missing")

In [91]:
# df.head()

In [92]:
df.to_csv("check.csv", index=0)

In [93]:
try:
    wdl=[c for c in df.columns if "WITHDRAW" in str(c).upper() ][0]
except:
    try:
        wdl=[c for c in df.columns if "AMOUNT" in str(c).upper() ][0]
    except: pass   

try:
    dep=[c for c in df.columns if "DEPOSIT" in str(c).upper() ][0]
except: pass

try:
    auto=[c for c in df.columns if "AUTOSWEEP" in str(c).upper() ][0]
except: pass

try:
    rev=[c for c in df.columns if "REVERSE" in str(c).upper() ][0]
except: pass

In [94]:
df[dep]=df[dep].apply( lambda x: x.split(' ')[0] if type(x) == str else x )

In [95]:
# df.head(20)

In [96]:
df[wdl]=df[wdl].apply( lambda x: x.split(' ')[0] if type(x) == str else x )                       

In [97]:
# df.head(20)

In [98]:
df[wdl]=df[wdl].astype(str).apply(lambda x: str(x).replace(",","").replace("Cr","").replace("Dr","")).astype(float) *-1

In [99]:
# df.head(20)

In [100]:
# df['Wdl1']=df['Wdl1'].fillna(df[dep].astype(str).apply(lambda x: str(x).replace(",","").replace("Cr","").replace("Dr","")).astype(float) ) 

In [101]:
# df.head(20)

In [102]:
df[dep]=df[dep].astype(str).apply(lambda x: str(x).replace(",","").replace("Cr","").replace("Dr","")).astype(float)

In [103]:
df["bal2"]=df[bal].astype(str).apply(lambda x:str(x).replace(",","").replace("Cr","").replace("Dr","")).astype(float)

In [106]:
df.tail(20)

2,Date,Particulars,Chq.No.,Withdrawals,Deposits,Autosweep,Reverse\rSweep,Balance(INR),bal,bal2
169,24-12-2019,MMT/IMPS/935817364617/Bno203ka15000rs/Narainpa...,NaN,-17238.0,0.0,NaN,NaN,,Dr,387248.88
170,24-12-2019,INF/INFT/021823226381/Bno669and672 /RAmlal,NaN,-28260.0,0.0,NaN,NaN,,Dr,415508.88
171,24-12-2019,MMT/IMPS/935818504223/Bno443/Atulgsmpr /SBIN00...,NaN,-13350.0,0.0,NaN,NaN,,Dr,428858.88
172,24-12-2019,MMT/IMPS/935818534786/Bno84dtd161119/Maharajap...,NaN,-22912.0,0.0,NaN,NaN,,Dr,451770.88
173,24-12-2019,MMT/IMPS/935819656217/Bno153/Parikhg /UCBA00...,NaN,-14650.0,0.0,NaN,NaN,,Dr,466420.88
174,26-12-2019,MMT/IMPS/935911889267/Total2lakh/Mahebderg /CO...,NaN,-115000.0,0.0,NaN,NaN,,Dr,581420.88
175,26-12-2019,662901701478: Rev Sweep From,NaN,NaN,NaN,0,"26,420.88",,Dr,555000.00
176,26-12-2019,CLG/GOYAL TRADERS/091490/OBC/24.12.2019,NaN,-0.0,21200.0,NaN,NaN,,Dr,533800.00
177,26-12-2019,CLG/SINDHI NAMKEEN/000491/HDF/25.12.2019,NaN,-0.0,35200.0,NaN,NaN,,Dr,498600.00
178,30-12-2019,MMT/IMPS/936412622220/Bno203clear/Narainpapa/H...,NaN,-15000.0,0.0,NaN,NaN,,Dr,513600.00


In [79]:
if "Cr" in df[bal][1]:
    print("true")

true


In [87]:
df[bal][1]

'6,35,658.12 Cr'

In [105]:
for i in df.index:
    print(i)
    if "Dr" in df[bal][i]:
        df[bal][i] = df[bal][i]*-1
    else:
        pass

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188


c:\users\mudracircle\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


TypeError: argument of type 'float' is not iterable

In [58]:
def isnan(value):
    try:
        return math.isnan(float(value))
    except:
        return False
    
for j,i in enumerate(df[rev]):
    if isnan(i) == False:
        df[dep][j] = df[rev][j]
    else:
        pass
    
for j,i in enumerate(df[auto]):
    if isnan(i) == False:
        df[wdl][j] = df[auto][j]
    else:
        pass

c:\users\mudracircle\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
c:\users\mudracircle\appdata\local\programs\python\python36\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
c:\users\mudracircle\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.o

In [59]:
df.to_csv("check.csv", index=0)